In [1]:
import numpy as np
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils,print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [3]:
data=pd.read_csv("/home/awis/Desktop/ML/data_sets/data.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
x=dataset
y=dataset
x=x[:, 0:1024]
y=y[:, 1024]

x_train=x[0:70000, :]
x_train=x_train/255.
x_test=x[70000:72001, :]
x_test=x_test/255.
print(x_train.shape)
# Reshape
y=y.reshape(y.shape[0],1)
y_train=y[0:70000, :]
y_train=y_train.T
y_test=y[70000:72001, :]
y_test=y_test.T

#print(y.shape)
#print(y_train.shape)
#print(y_test.shape)

(70000, 1024)


In [4]:
print("Number of training examples = "+str(x_train.shape[0]))
print("Number of test samples = " + str(x_test.shape[0]))
print("x_train shape: "+ str(x_train.shape))
print("y_train shape: "+ str(y_train.shape))
print("x_test shape: "+ str(x_test.shape))
print("y_test shape: "+ str(y_test.shape))

Number of training examples = 70000
Number of test samples = 2000
x_train shape: (70000, 1024)
y_train shape: (1, 70000)
x_test shape: (2000, 1024)
y_test shape: (1, 2000)


In [5]:
image_x=32
image_y=32

train_y=np_utils.to_categorical(y_train) # feature vector conversion
test_y=np_utils.to_categorical(y_test)   # feature vector conversion
#print(train_y)
#print(train_y.shape)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
x_train=x_train.reshape(x_train.shape[0],image_x,image_y, 1)
x_test=x_test.reshape(x_test.shape[0],image_x,image_y, 1)

In [6]:
print("X_train shape: " + str(x_train.shape))
print("Y_train shape: " + str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [7]:
def keras_model(image_x,image_y):
    num_of_classes = 37
    model = Sequential() #sequential model
    model.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "devanagari.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list

In [8]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(x_train, train_y, validation_data=(x_test, test_y), epochs=8, batch_size=64,callbacks=callbacks_list)
scores = model.evaluate(x_test, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)
model.save('devanagari.h5')

Train on 70000 samples, validate on 2000 samples
Epoch 1/8
70000/70000 [==============================] - 113s 2ms/step - loss: 0.7758 - accuracy: 0.7794 - val_loss: 0.3293 - val_accuracy: 0.9005

Epoch 00001: val_accuracy improved from -inf to 0.90050, saving model to devanagari.h5
Epoch 2/8
70000/70000 [==============================] - 116s 2ms/step - loss: 0.2331 - accuracy: 0.9290 - val_loss: 0.1999 - val_accuracy: 0.9445

Epoch 00002: val_accuracy improved from 0.90050 to 0.94450, saving model to devanagari.h5
Epoch 3/8
70000/70000 [==============================] - 114s 2ms/step - loss: 0.1505 - accuracy: 0.9543 - val_loss: 0.1511 - val_accuracy: 0.9540

Epoch 00003: val_accuracy improved from 0.94450 to 0.95400, saving model to devanagari.h5
Epoch 4/8
70000/70000 [==============================] - 116s 2ms/step - loss: 0.1121 - accuracy: 0.9656 - val_loss: 0.1317 - val_accuracy: 0.9650

Epoch 00004: val_accuracy improved from 0.95400 to 0.96500, saving model to devanagari.h5
Ep